# Generate a PPi train + validation set from [APID](http://cicblade.dep.usal.es:8080/APID/init.action) and a Y2H test set from [HuRI](http://www.interactome-atlas.org/download)
## prep

- March 2021 version of [APID](http://cicblade.dep.usal.es:8080/APID/init.action)
- interactomes for the 35 organisms listed as "with more than 500 interactions" only
- some organism datasets will be dropped later, so not fetching smaller sets is ok
- binary interactomes -> Level 2 -> `Q1` suffix
- not using *H. sapiens* for training -> `9606_*Q1.txt`


first number is number of IDs, second is number of PPIs

| APID     |              ISI |            noISI |      loss |
|---------:|:----------------:|:----------------:|:---------:|
| w/ Human | 67'240 : 319'429 | 61'429 : 296'653 | .09 : .07 |
| no Human | 55'014 : 179'349 | 45'026 : 161'598 | .18 : .10 |
|     loss |    .18 : .44     |    .27 : .46     | .33 : .49 |

In [ ]:
from __future__ import annotations

import os
import sys
import shutil
import importlib

from pathlib import Path
from dataclasses import dataclass, field
from dataclass_wizard import JSONWizard

In [ ]:
!which python
!pwd

In [ ]:
# assert False, 'don\'t run'
# !mv apid.zip ../apid.zip
# !mv hi_union.psi ../hi_union.psi
# !ln ../apid.zip apid.zip
# !ln ../hi_union.psi hi_union.psi

In [ ]:
cwd = Path.cwd()
package_root = str(cwd.parent.parent)
if package_root not in sys.path:
    sys.path.append(package_root)

In [ ]:
session = '6072'

shutil.copytree(cwd, cwd.with_name(session),
                symlinks=True, dirs_exist_ok=True)
os.chdir(cwd.with_name(session).resolve())
!pwd

In [ ]:
from data.utils import api, extract, general, pairs, plot, reduce

from data.utils.api import *
from data.utils.extract import *
from data.utils.general import *
from data.utils.pairs import *
from data.utils.plot import *
from data.utils.reduce import *

In [ ]:

@dataclass
class Config(JSONWizard):
    class _(JSONWizard.Meta):
        key_transform_with_dump = 'SNAKE'

    keep_human: bool = False
    keep_interspecies: bool = False
    keep_homodimers: bool = True
    rostclust_pretend: bool = False

    ap: Path = Path('apid_sequences')
    hp: Path = Path('huri_sequences')
    qp: Path = None

    min_seq_len: int = 50
    max_seq_len: int = 1500

    psi_path = Path('hi_union.psi')
    hval_config: dict = field(default_factory=dict)

    seed: int = 64
    ratio: float = 10.0
    strategy: SamplingStrategy = SamplingStrategy.BALANCED

    val_set_size: float = .1
    val_species: Set[str] = field(default_factory=set)

    val_raw_fasta: Path = ap / 'apid_validation_raw.fasta'
    val_raw_tsv: Path = ap / 'apid_validation_raw.tsv'
    val_rr_fasta: Path = ap / 'apid_validation_rr.fasta'
    val_rr_tsv: Path = ap / 'apid_validation_rr.tsv'
    val_c3_fasta: Path = ap / 'apid_validation_c3.fasta'
    val_fasta: Path = Path('apid_validation.fasta')
    val_tsv: Path = Path('apid_validation.tsv')

    test_raw_fasta: Path = hp / 'huri_test_raw.fasta'
    test_raw_tsv: Path = hp / 'huri_test_raw.tsv'
    test_rr_fasta: Path = hp / 'huri_test_rr.fasta'
    test_rr_tsv: Path = hp / 'huri_test_rr.tsv'
    test_c3_fasta: Path = hp / 'huri_test_c3.fasta'
    test_fasta: Path = Path('huri_test.fasta')
    test_tsv: Path = Path('huri_test.tsv')

    train_tsv: Path = Path('apid_train.tsv')
    train_fasta: Path = Path('apid_train.fasta')
    train_positives: Path = ap / 'apid_train_positives.tsv'

    # results
    weird_species: set[int] = field(default_factory=set)
    train_bias: str = None
    train_seqs: int = None
    train_size: int = None
    test_seqs: int = None
    val_seqs: int = None
    val_bias: str = None
    val_size: int = None
    val_sizes: str = None
    test_bias: float = None
    test_size: int = None
    test_sizes: str = None


val_species = {39947}
hval_config = dict(shortAlignmentLen=50,
                   longSeqLen=180,
                   reduceRandomAlign=False,
                   clusterThreshold=20)

c = Config(hval_config=hval_config, val_species=val_species)

In [ ]:
importlib.reload(api)
importlib.reload(extract)
importlib.reload(general)
importlib.reload(pairs)
importlib.reload(plot)
importlib.reload(reduce)

from data.utils.api import *
from data.utils.extract import *
from data.utils.general import *
from data.utils.pairs import *
from data.utils.plot import *
from data.utils.reduce import *

## Toy example
#### bias of negative dataset

In [ ]:
# demonstrate sampling strategies: toy PPI table example
raw = '''A\tB\nA\tC\nC\tF\nA\tE\nK\tL\nC\tK\nB\tG\nG\tM\nA\tO\nE\tO\nI\tJ\nJ\tK\nI\tK'''
ppis = ppis_from_string(raw)
ppi_dict = dict()
ppis

In [ ]:
ppi_dict['balanced'], _ = find_negative_pairs(ppis, ratio=1.5, seed=42,
                                              strategy=SamplingStrategy.BALANCED)
# balanced sampling tries to conserve the distribution of degrees -> high-bias dataset

In [ ]:
ppi_dict['random'], _ = find_negative_pairs(ppis, ratio=1.5, seed=42,
                                            strategy=SamplingStrategy.RANDOM)
# random sampling gives a nearly un-biased dataset

In [ ]:
ppi_dict['complement'], _ = find_negative_pairs(ppis, ratio=10, seed=42,
                                                strategy=SamplingStrategy.BALANCED)
# sampling the entire network complement will perfectly invert the degree distribution

#### strategies visualized
It becomes apparent that the combination of graph size and density is important, and that random or balanced sampling generate different sets of negatives.

In [ ]:
_ = draw_toy_ppis(ppis, ppi_dict)

#### with homodimers

In [ ]:
# added F-F
raw = '''A\tB\nA\tC\nC\tF\nA\tE\nK\tL\nC\tK\nB\tG\nG\tM\nA\tO\nE\tO\nI\tJ\nJ\tK\nI\tK\nF\tF'''
ppis = ppis_from_string(raw)

In [ ]:
ppi_dict = dict()
ppi_dict['balanced'], _ = find_negative_pairs(ppis, ratio=1.5, seed=42,
                                              strategy=SamplingStrategy.BALANCED)
ppi_dict['random'], _ = find_negative_pairs(ppis, ratio=1.5, seed=42,
                                            strategy=SamplingStrategy.RANDOM)
ppi_dict['complement'], _ = find_negative_pairs(ppis, ratio=10, seed=42,
                                                strategy=SamplingStrategy.BALANCED)
_ = draw_toy_ppis(ppis, ppi_dict)

#### input homodimers, but not allowed as negatives

In [ ]:
ppi_dict = dict()
ppi_dict['balanced'], _ = find_negative_pairs(ppis, ratio=1.5, seed=42,
                                              strategy=SamplingStrategy.BALANCED,
                                              homodimers=False)
ppi_dict['random'], _ = find_negative_pairs(ppis, ratio=1.5, seed=42,
                                            strategy=SamplingStrategy.RANDOM,
                                            homodimers=False)
ppi_dict['complement'], _ = find_negative_pairs(ppis, ratio=10, seed=42,
                                                strategy=SamplingStrategy.BALANCED,
                                                homodimers=False)
_ = draw_toy_ppis(ppis, ppi_dict)

## Train set: APID
#### extract from `<taxid>*_Q1.txt` files

In [ ]:
c.qp = unzip_apid(zip_path='apid.zip', keep_human=c.keep_human,
                  keep_interspecies=c.keep_interspecies)
c.qp

In [ ]:
uniprot_ids = extract_apid_uniprot_ids(c.qp)
len(uniprot_ids)

In [ ]:
ppis = extract_apid_ppis(c.qp)
len(ppis)

In [ ]:
ppis.head()

#### interspecies interactions
Check if there are PPIs (based on UniProt IDs) that occur in several species. We use APID `noISI` data per default, so the exclusion of this data is not implemented.

In [ ]:
find_multi_species_ppis = lambda ppi_df: pd.concat(
    [df for i, df in ppi_df.groupby(
        ['UniprotID_A', 'UniprotID_B']) if len(df) > 1]
    + [ppi_df.loc[ppi_df.species == 'is there a marsupilami?']])

find_multi_species_ppis(ppis)

In [ ]:
# proof that actually worked: add some dummy data
for i in [0, 1]:
    s = ppis.iloc[i].copy()
    s.species = 'marsupilami'
    ppis = pd.concat([ppis, pd.DataFrame(s).T], axis=0)

marsu = find_multi_species_ppis(ppis)

# drop the dummy data
ppis = ppis.loc[ppis.species != 'marsupilami']
marsu

or more generally protein IDs that occur in several species:

In [ ]:
find_multi_species_proteins(ppis)

#### species lookup

In [ ]:
# pick a charismatic animal for validation!
taxonomy = fetch_taxonomic_info(ppis.species)
taxonomy

#### download sequences

In [ ]:
# # if you are re-running this without changes above or deleting files, skip!
# _ = uniprot_api_fetch(uniprot_ids, c.ap / 'apid')

#### map IDs to seq hashes

In [ ]:
hash_ppis = ppis_to_hashes(ppis, c.ap / 'apid.json').drop_duplicates()
hash_ppis.head()

In [ ]:
len(ppis) - len(hash_ppis)

In [ ]:
fasta = SeqIO.to_dict(SeqIO.parse(c.ap / 'apid.hash.fasta', 'fasta'))
len(fasta)

#### filter by seq length

In [ ]:
c.min_seq_len, c.max_seq_len

In [ ]:
_hash_ppis, _fasta, fig = filter_ppis_and_fasta_by_len(
    hash_ppis, fasta, c.min_seq_len, c.max_seq_len)
len(fasta), len(_fasta)

continue if these limits were ok, or change/set the values for
`cfg.min_seq_len, cfg.max_seq_len` and re-run the filtering cell

In [ ]:
hash_ppis = _hash_ppis
fasta = _fasta

#### homodimer share
Overall count, share, overall PPI count

In [ ]:
_ = plot_homodimer_share(hash_ppis)
c.keep_homodimers, *count_homodimers(hash_ppis),

In [ ]:
if not c.keep_homodimers:
    hash_ppis = drop_homodimers(hash_ppis)
    hash_ppis, fasta = shrink_both_ways(hash_ppis, fasta)
    len(hash_ppis), len(fasta)

## Train set: negatives

In [ ]:
c.ratio, c.seed, c.strategy

In [ ]:
train_ppis, train_negs, train_bias = make_train_negatives(
    hash_ppis, ratio=c.ratio, seed=c.seed, strategy=c.strategy, homodimers=c.keep_homodimers)

In [ ]:
train_bias

#### species without negatives or a constant Pearson/Spearman

In [ ]:
train_bias.loc[train_bias.bias.isna()]

In [ ]:
weird_species = set(train_bias.loc[train_bias.bias.isna(), 'species'])
train_ppis.loc[train_ppis['species'].isin(weird_species)]

In [ ]:
train_negs.loc[train_negs['species'].isin(weird_species)]

This is very little data that we can safely drop. For curiosity's sake:

In [ ]:
taxonomy.loc[taxonomy.species.isin(weird_species)]

It's actually ok to just drop these organisms at this point - no need to backpropagate this farther up.

In [ ]:
# actually drop the data here
train_ppis, train_negs, train_bias = [df.loc[~df.species.isin(weird_species)]
                                      for df in (train_ppis, train_negs, train_bias)]
c.weird_species = weird_species
len(train_ppis)

#### set size and bias
This is *before* the extraction of the validation set and therefor preliminary.

In [ ]:
_ = plot_bias(train_ppis, train_negs, train_bias, c.ratio)

## Validation set: APID
#### val: select species/proteins
Either by sampling a subset of proteins and selecting the PPIs or picking the ones for one or more particular species

In [ ]:
c.val_set_size, c.val_species

In [ ]:
# # just a suggestion
# val_species = {7955, 9031, 9913, 10377, 37296, 39947}

In [ ]:
train_ppis['label'] = 1
if c.val_species is not None:
    train, val = make_validation_species(train_ppis, c.val_species)
    train_negs = train_negs.loc[~train_negs.species.isin(c.val_species)]
else:
    # if the set becomes too large, try another seed! With this approach, PPIs *will* be lost
    train, val = make_validation_split(train_ppis, c.val_set_size, c.seed)
len(train), len(train) / len(train_ppis), len(val), len(val) / len(train_ppis)

In [ ]:
val.head()

In [ ]:
val.species.unique()

#### train: select / re-sample negatives
We have already generated negatives above. If we want to use them, we have to drop the negatives with a protein not in the `train` PPIs, then take the rest as training set negatives. But let's also look at the bias of this cropped set! (Resample negatives for `val` later, after redundancy reduction.)

In [ ]:
train_proteins = set(np.unique(train.iloc[:, [0, 1]]))
_train_negs = train_negs.loc[(train_negs.hash_A.isin(train_proteins))
                             & (train_negs.hash_B.isin(train_proteins))]
len(_train_negs), len(train_negs), len(_train_negs) / len(train_negs)

In [ ]:
estimate_bias(train, _train_negs, corrtype=CorrelationType.SPEARMAN)

In [ ]:
estimate_bias(train, _train_negs, corrtype=CorrelationType.PEARSON)

In [ ]:
fig = plot_bias(train, _train_negs, train_bias, c.ratio)
fig.savefig('train_bias_0_og.png', transparent=False)

Redraw a set of negative anyway to see if it would fit significantly better:

In [ ]:
t_ppis, t_negs, t_bias = make_train_negatives(train, c.strategy, c.ratio, c.seed, c.keep_homodimers)

In [ ]:
# drop na data
t_ppis = t_ppis.loc[~t_ppis.species.isin(t_bias.loc[t_bias.bias.isna()].species)]

In [ ]:
fig = plot_bias(t_ppis, t_negs, t_bias, c.ratio)
fig.savefig('train_bias_1_resample.png', transparent=False)

In [ ]:
# pick one!
picked_idx = 1
train_all = [pd.concat((train, _train_negs)), pd.concat((t_ppis, t_negs))][picked_idx]
c.train_bias = [train_bias.to_json(), t_bias.to_json()][picked_idx]
len(train_all)

Separate the validation set sequences from the ones remaining as train set

In [ ]:
_, train_seqs = shrink_both_ways(train_all.copy(), fasta.copy())
c.train_seqs = len(train_seqs)
len(train_seqs), len(fasta)

#### train: save


In [ ]:
_ = SeqIO.write(values_sorted_by_key(train_seqs), c.train_fasta, 'fasta')
train_all.to_csv(c.train_tsv, sep='\t', header=True, index=False)
c.train_size = len(train_all)
c.train_fasta, c.train_tsv

#### val: save

In [ ]:
_, val_seqs = shrink_both_ways(val.copy(), fasta.copy())
len(val_seqs), len(fasta)

Mind that the TSV will contain only positives! Sample negatives only after the whole `uniqueprot`-shebang.

In [ ]:
_ = SeqIO.write(values_sorted_by_key(val_seqs), c.val_raw_fasta, 'fasta')
val.to_csv(c.val_raw_tsv, sep='\t', header=True, index=False)
c.val_raw_fasta, c.val_raw_tsv

## Test set: HuRI
#### download and extract PPIs

In [ ]:
c.psi_path

In [ ]:
# it's somehow a lot faster to download this from a browser!
# or run !wget "http://www.interactome-atlas.org/data/HI-union.psi" -O "hi_union.psi"
download_y2h_interactome(c.psi_path)

In [ ]:
huri_ppis, huri_seq_ids = extract_huri_ppis(c.psi_path)
len(huri_ppis)

In [ ]:
huri_seqs, _ = fetch_huri_seqs(huri_seq_ids, c.hp / 'huri')
len(huri_seqs)

#### map + filter PPIs

In [ ]:
print(set(huri_ppis.species))  # only human, but keep the label
huri_ppis.head()

In [ ]:
huri_hash_ppis = ppis_to_hashes(huri_ppis, c.hp / 'huri.json')
len(huri_ppis) - len(huri_hash_ppis)

In [ ]:
huri_hash_ppis = remove_ids_from(huri_hash_ppis, black_list_fasta=c.train_fasta)
len(huri_hash_ppis)

In [ ]:
*count_homodimers(huri_hash_ppis), c.keep_homodimers

In [ ]:
if not c.keep_homodimers:
    huri_hash_ppis = drop_homodimers(huri_hash_ppis)
    huri_hash_ppis, huri_seqs = shrink_both_ways(huri_hash_ppis, huri_seqs)
    len(huri_hash_ppis), len(huri_seqs)

In [ ]:
_huri_hash_ppis, _huri_seqs, fig = filter_ppis_and_fasta_by_len(
    huri_hash_ppis, huri_seqs, c.min_seq_len, c.max_seq_len)
len(huri_seqs), len(_huri_seqs)

#### save to files

In [ ]:
huri_hash_ppis = _huri_hash_ppis
huri_seqs = _huri_seqs
_ = SeqIO.write(huri_seqs.values(), c.test_raw_fasta, 'fasta')
huri_hash_ppis.to_csv(c.test_raw_tsv, sep='\t', header=True, index=False)
c.test_raw_fasta, c.test_raw_tsv

## Redundancy Reduction with [rostclust](https://git.rostlab.org/bernhoferm/mmseqs2-uniqueprot/-/tree/F_typer_app)

**Don't run the next cells in the wrong order, and don't forget to copy the files back and forth if you are running `rostclust` elsewhere! <br>In particular, copy files after `shrink_files_both_ways`. Was that name not clear?**

In [ ]:
c.hval_config

#### redundancy-reduce test and validation
The flag below allows me to test this notebook without the MMseqs2 runs. Leave `rostclust_pretend` at `True` and copy the two result files to this folder; or run `rostclust uniqueprot[2D]` now and here.

In [ ]:
c.rostclust_pretend

In [ ]:
if c.rostclust_pretend:
    !rsync -zar -e ssh "." "kaindll@lapislazuli:/home/k/kaindll/ppi/data/ppi_dataset"

In [ ]:
# redundancy-reduce the test set
run_uniqueprot(c.test_raw_fasta, c.test_rr_fasta, c.hval_config, pretend=c.rostclust_pretend)

In [ ]:
# and validation
run_uniqueprot(c.val_raw_fasta, c.val_rr_fasta, c.hval_config, pretend=c.rostclust_pretend)

In [ ]:
if c.rostclust_pretend:
    !rsync -zar -e ssh "kaindll@lapislazuli:/home/k/kaindll/ppi/data/ppi_dataset/" "."

In [ ]:
fastalen = lambda _file: len([r.id for r in SeqIO.parse(_file, 'fasta')])

In [ ]:
# also drop PPIs accordingly
assert c.test_rr_fasta.is_file()
shrink_files_both_ways(c.test_raw_tsv, c.test_rr_fasta,
                       new_tsv_name=c.test_rr_tsv,
                       new_fasta_name=c.test_fasta)  # finished!
c.test_seqs = fastalen(c.test_fasta)
assert c.val_rr_fasta.is_file()
shrink_files_both_ways(c.val_raw_tsv, c.val_rr_fasta,
                       new_tsv_name=c.val_rr_tsv,
                       new_fasta_name=c.val_fasta)  # finished!
c.val_seqs = fastalen(c.val_fasta)

In [ ]:
if c.rostclust_pretend:
    !rsync -zar -e ssh "." "kaindll@lapislazuli:/home/k/kaindll/ppi/data/ppi_dataset"

#### cluster `test` and `val` against `train`

First, cluster the `HuRI` test set against the `APID` train set to get **C1-3** labels for the final test PPIs. Then, cluster the `APID` validation set against the `APID` train set, which gives us **C1-3** labels for the validation set.

In [ ]:
run_uniqueprot2D(c.test_fasta, c.train_fasta, c.test_c3_fasta,
                 hval_config=c.hval_config, pretend=c.rostclust_pretend)

In [ ]:
run_uniqueprot2D(c.val_fasta, c.train_fasta, c.val_c3_fasta,
                 hval_config=c.hval_config, pretend=c.rostclust_pretend)

In [ ]:
if c.rostclust_pretend:
    !rsync -zar -e ssh "kaindll@lapislazuli:/home/k/kaindll/ppi/data/ppi_dataset/" "."

In [ ]:
assert c.test_c3_fasta.is_file()
assert c.val_c3_fasta.is_file()

## Generate negatives

Negatives are sampled *per species*. Mind that the test set is human-only!
Also, identify and assign **C1/C2/C3** class labels at this point.

In [ ]:
c.ratio, c.seed, c.strategy

#### APID val: generate & classify

In [ ]:
# sanity check if there are any proteins in test that
# were already in train. re: "do not add back in identical ones"
assert not ({r.id for r in SeqIO.parse(c.train_fasta, 'fasta')} & {
    r.id for r in SeqIO.parse(c.test_fasta, 'fasta')})

In [ ]:
for p in [c.val_rr_tsv, c.val_c3_fasta, c.val_fasta]:
    assert p.is_file(), f'missing {p}'

In [ ]:
val_ppis, val_negatives, val_bias = make_test_negatives(
    c.val_rr_tsv, c.val_c3_fasta, c.val_fasta, c.strategy,
    c.ratio, c.seed, c.keep_homodimers)
c.val_bias = str(val_bias) if type(val_bias) == np.float64 else val_bias.to_json()
val_negatives.head()

In [ ]:
val_ppis.head()

In [ ]:
if type(val_bias) != np.float64:
    fig = plot_bias(val_ppis, val_negatives, val_bias, c.ratio)
    fig.savefig('val_bias.png', transparent=False)

In [ ]:
val_all = pd.concat([val_ppis, val_negatives])
c.val_size = len(val_all)
len(val_all)

In [ ]:
with mpl.rc_context({'figure.dpi': 100}):
    fig = plot_c_classes(val_all)
    c.val_sizes = val_all[['cclass', 'label']].value_counts().to_json()
    fig.savefig('val_cclasses.png', transparent=False)

In [ ]:
# have a direct look at the (ranked) degrees, but only for some species
n_degrees_per_sp = fetch_degrees(val_all).groupby('species')[
    'x'].size().sort_values(ascending=False)
# pick two larger and two smaller species sets
sps = list(n_degrees_per_sp.index)[:2] + list(n_degrees_per_sp.index)[-5:-3]
print(sps)
n_degrees_per_sp

In [ ]:
with mpl.rc_context({'figure.dpi': 72 if len(sps) > 2 else 200}):
    # can't unFUBAR the axis limits ...
    fig = plot_test_degrees(val_all.loc[val_all.species.isin(sps)], c.ratio, flip=False)
    fig.savefig('val_degrees.png', transparent=False)

#### APID val: save TSV
The `FASTA` was finalized already, right after the redundancy reduction and before clustering against `train`.

In [ ]:
val_all.to_csv(c.val_tsv, sep='\t', header=True, index=False)
c.val_tsv

#### HuRI test: generate & classify

In [ ]:
for p in [c.test_rr_tsv, c.test_c3_fasta, c.test_fasta]:
    assert p.is_file(), f'missing {p}'

In [ ]:
test_ppis, test_negatives, test_bias = make_test_negatives(
    c.test_rr_tsv, c.test_c3_fasta, c.test_fasta, c.strategy,
    c.ratio, c.seed, c.keep_homodimers)
c.test_bias = test_bias
test_negatives.head()

In [ ]:
test_ppis.head()

In [ ]:
test_all = pd.concat([test_ppis, test_negatives])
c.test_size = len(test_all)
len(test_all)

In [ ]:
with mpl.rc_context({'figure.dpi': 100}):
    fig = plot_c_classes(test_all)
    c.test_sizes = test_all[['cclass', 'label']].value_counts().to_json()
    fig.savefig('test_cclasses.png', transparent=False)

In [ ]:
fig = plot_test_degrees(test_all, c.ratio)
fig.savefig('test_degrees.png', transparent=False)

#### HuRI test: save TSV

In [ ]:
test_all.to_csv(c.test_tsv, sep='\t', header=True, index=False)
c.test_tsv

---

In [ ]:
with Path(f'cfg.json').open('w') as json_file:
    json_file.write(c.to_json(indent=2))
print(c.to_json(indent=2))